# COVID Cross Reference
Find cross-reactive (same) TCR sequences for ImmuneCODE(MIRA) database (SARS-CoV2) and other viruses in VDJdb:
<ul>
    <li>M1: GILGFVFTL</li>
    <li>CMV(pp65): NLVPMVATV </li>
    <li>MART-1: ELAGIGILTV </li>
</ul>

SARS-CoV2 epitopes are from TSCAN (their gene region and occurrences in ImmuneCODE):

|Epitope|Gene Region|Occurrences|
|-----|-------|------|
|KLWAQCVQL|ORF1ab|741|
|YLQPRTFLL|S|1138|
|ALWEIQQVV|ORF1ab|5|
|LLLDRLNQL|N|1028|


## Define functions and constants

In [39]:
import numpy as np
import pandas as pd
import scipy.sparse
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import sparse
import warnings
import time
import plotly

# Functions for data
def _read_raw_data(file_path):
    file_type = file_path[-3:]
    assert file_type == 'csv' or file_type == 'tsv', "The input file must be csv or tsv file."
    if file_type == 'csv':
        df = pd.read_csv(file_path)
    else:
        df = pd.read_csv(file_path, sep='\t')
    return df

def read_data(file_path):
    raw_data_df = _read_raw_data(file_path)
    raw_data_columns = raw_data_df.columns.values.tolist()
    assert 'epitope' in raw_data_columns and 'cdr3b' in raw_data_columns, "Columns 'Epitope' and 'CDR3' must in the input file."
    data_df = pd.DataFrame(raw_data_df[['epitope', 'cdr3b']])
    return data_df


def read_data_MIRA(file_path):
    raw_data_df = _read_raw_data(file_path)
    raw_data_columns = raw_data_df.columns.values.tolist()
    assert 'TCR BioIdentity' in raw_data_columns and 'Amino Acids' in raw_data_columns, "Columns 'Epitope' and 'CDR3' must in the input file."
    data_df = pd.DataFrame(
        raw_data_df[['TCR BioIdentity', 'Amino Acids', 'Experiment', 'Start Index in Genome', 'End Index in Genome',
                     'ORF Coverage']])
    return data_df


def read_data_Metadata(file_path):
    raw_data_df = _read_raw_data(file_path)
    raw_data_columns = raw_data_df.columns.values.tolist()
    assert 'Experiment' in raw_data_columns and 'HLA-A-1' in raw_data_columns and 'HLA-A-2' in raw_data_columns, \
        "Columns 'Experiment' , 'HLA-A-1' and 'HLA-A-2' must in the input file."
    data_df = pd.DataFrame(
        raw_data_df[['Experiment', 'HLA-A-2', 'HLA-A-2']])
    return data_df


def read_data_VDJdb(file_path):
    raw_data_df = _read_raw_data(file_path)
    raw_data_columns = raw_data_df.columns.values.tolist()
    assert 'Gene' in raw_data_columns and 'CDR3' in raw_data_columns, \
        "Gene' and 'CDR3' must in the input file."
    data_df = pd.DataFrame(
        raw_data_df[['Gene', 'CDR3', 'V', 'J']])
    data_df = data_df.rename(columns={'CDR3': 'cdr3'})
    return data_df


def arrange_MIRA(data_df):
    data_df[['cdr3b', 'V', 'J']] = data_df['TCR BioIdentity'].str.split('+', expand=True)
    data_df = data_df.drop(columns=["TCR BioIdentity"])
    data_df = data_df.rename(columns={'Amino Acids': 'epitope'})
    return data_df

TSCAN_epitope = ['LLLDRLNQL','YLQPRTFLL','KLWAQCVQL','ALWEIQQVV']

## Read MIRA data

### Rearrangments and removal of duplicates

 Format is ['epitope', 'Experiment', 'Start Index in Genome', 'End Index in Genome', 'cdr3b', 'V', 'J']

In [4]:
MIRA_data_folder = Path('./Data')
MIRA_pathlib_ci = MIRA_data_folder / 'peptide-detail-ci.csv'
MIRA_path_ci = str(MIRA_pathlib_ci)
MIRA_pathlib_cii = MIRA_data_folder / 'peptide-detail-cii.csv'
MIRA_path_cii = str(MIRA_pathlib_cii)
# Read and arrange data
data_raw_ci = read_data_MIRA(MIRA_path_ci)
data_raw_cii = read_data_MIRA(MIRA_path_cii)
data_ci = arrange_MIRA(data_raw_ci)
data_cii = arrange_MIRA(data_raw_cii)

In [6]:
# Drop Duplicates
data_ci = data_ci.drop_duplicates(ignore_index=True)
data_cii = data_cii.drop_duplicates(ignore_index=True)  
data_ci['MHC'] = 'I'
data_cii['MHC'] = 'II'
# Combine raw data and drop duplicates
data_all_raw = pd.concat([data_ci, data_cii], ignore_index=True)
MIRA_data = data_all_raw.drop_duplicates(ignore_index=True)

### View the table format

In [7]:
MIRA_data.shape

(156522, 9)

In [10]:
MIRA_data.head(5)

,epitope,Experiment,Start Index in Genome,End Index in Genome,ORF Coverage,cdr3b,V,J,MHC
0,"ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",eAV93,533,24073,"ORF1ab,surface glycoprotein",CASSAQGTGDRGYTF,TCRBV27-01,TCRBJ01-02,I
1,"ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",eOX56,533,24073,"ORF1ab,surface glycoprotein",CASSLVATGNTGELFF,TCRBV07-09,TCRBJ02-02,I
2,"ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",eAV93,533,24073,"ORF1ab,surface glycoprotein",CASSKGTVSGLSG,TCRBV21-01,TCRBJ02-07,I
3,"ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",eQD124,533,24073,"ORF1ab,surface glycoprotein",CALKVGADTQYF,TCRBV30-01,TCRBJ02-03,I
4,"ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",eAV93,533,24073,"ORF1ab,surface glycoprotein",CASSLWASGRGGTGELFF,TCRBV27-01,TCRBJ02-02,I


## Cross Reference with M1
### Read M1 data

In this dataset, the M1 epitope is GILGFVFTL. All cdrs are from HomoSapiens (human).
All are HLA-A02 cdrs. All are MHC class I.
The format is ['Gene', 'cdr3', 'V', 'J']

In [12]:
M1_data_folder = Path('./Data')
M1_pathlib = M1_data_folder / 'VDJdb_M1_TCR.tsv'
M1_raw = read_data_VDJdb(str(M1_pathlib))
M1_raw = M1_raw.drop_duplicates(ignore_index=True)
# Rearrange M1_data
M1_data = M1_raw.loc[M1_raw['Gene'] == 'TRB'] 
M1_data = M1_data.rename(columns={'cdr3': 'cdr3b'})
M1_cdr3b = M1_data[['cdr3b']]
M1_cdr3b = M1_cdr3b.drop_duplicates(ignore_index=True) 

In [18]:
M1_cdr3b.shape

(3407, 1)

In [19]:
M1_cdr3b.head(5)

,cdr3b
0,CASSSRSSYEQYF
1,CASSSRASYEQYF
2,CASSTRGAYEQYF
3,CASSSRSAYEQYF
4,CSVLQGSPYEQYF


### Find shared CDR3b of M1 with COVID-19

This will extract entries with same cdr3b from MIRA dataset 

In [15]:
M1_Common = pd.DataFrame(columns=data_all.columns.values.tolist())
for index, row in MIRA_data.iterrows():
    if row['cdr3b'] in M1_cdr3b['cdr3b'].values:
        M1_Common = M1_Common.append(row, ignore_index=True)

In [55]:
len(pd.unique(M1_Common['cdr3b']))

276

In [56]:
M1_Common.head(5)

,epitope,Experiment,Start Index in Genome,End Index in Genome,ORF Coverage,cdr3b,V,J,MHC
0,"APKEIIFL,KEIIFLEGETL",eEE226,2468,2506,ORF1ab,CASSYTLNTEAFF,TCRBV27-01,TCRBJ01-01,I
1,"APKEIIFL,KEIIFLEGETL",eAV93,2468,2506,ORF1ab,CASSSGNTEAFF,TCRBV05-01,TCRBJ01-01,I
2,"APKEIIFL,KEIIFLEGETL",eEE226,2468,2506,ORF1ab,CASGGTSTDTQYF,TCRBV12-05,TCRBJ02-03,I
3,"APKEIIFL,KEIIFLEGETL",eXL30,2468,2506,ORF1ab,CASSHGAEAFF,TCRBV03-01/03-02,TCRBJ01-01,I
4,KPLEFGATSAAL,eOX52,3137,3172,ORF1ab,CASSIVSNQPQHF,TCRBV19-01,TCRBJ01-05,I


### Write to file

In [20]:
M1_Common.to_csv(str(MIRA_data_folder / "Output/M1_Common.csv"), index=False)

### Appearance of TSCAN epitopes in shared CDR3b binding targets

In [27]:
M1_Common[M1_Common['epitope'].str.contains('LLLDRLNQL') == True]

,epitope,Experiment,Start Index in Genome,End Index in Genome,ORF Coverage,cdr3b,V,J,MHC
534,"ALALLLLDR,GDAALALLLL,LALLLLDRL,LLLDRLNQL,LLLLD...",eHO136,28916,28963,nucleocapsid phosphoprotein,CASSQGNYGYTF,TCRBV14-01,TCRBJ01-02,I
535,"ALALLLLDR,GDAALALLLL,LALLLLDRL,LLLDRLNQL,LLLLD...",eOX49,28916,28963,nucleocapsid phosphoprotein,CASSQGNYGYTF,TCRBV03-01/03-02,TCRBJ01-02,I
536,"ALALLLLDR,GDAALALLLL,LALLLLDRL,LLLDRLNQL,LLLLD...",eHO140,28916,28963,nucleocapsid phosphoprotein,CASGLPYEQYF,TCRBV27-01,TCRBJ02-07,I
577,"NGGDAALALLLLDRLNQLE,QASSRSSSRSRNSSRNSTP,RSRNSS...",eAV100,28814,28966,nucleocapsid phosphoprotein,CASSPRSSYEQYF,TCRBV18-01,TCRBJ02-07,II
578,"NGGDAALALLLLDRLNQLE,QASSRSSSRSRNSSRNSTP,RSRNSS...",eNL187,28814,28966,nucleocapsid phosphoprotein,CASSLGTSYEQYF,TCRBV05-05,TCRBJ02-07,II


In [31]:
M1_Common[M1_Common['epitope'].str.contains('LLLDRLNQL') == True].shape[0]

5

In [41]:
def search_TSCAN(Common_data,TSCAN_epitope):
    table = np.zeros((len(TSCAN_epitope),1),dtype = int)
    for index,epitope in enumerate(TSCAN_epitope):
        table[index][0] = Common_data[Common_data['epitope'].str.contains(epitope) == True].shape[0]
    return(table)

In [43]:
pd.DataFrame(search_TSCAN(M1_Common,TSCAN_epitope),columns = ['Frequency'],index = TSCAN_epitope)

,Frequency
LLLDRLNQL,5
YLQPRTFLL,0
KLWAQCVQL,0
ALWEIQQVV,0


## Cross Reference with CMV

### Read CMV data
In this CMV dataset, epiotpe is only NLVPMVATV. All cdrs are from HomoSapiens (human).
All are HLA-A02 cdrs. All are MHC class I.
format is ['Gene', 'cdr3', 'V', 'J']

In [45]:
CMV_data_folder = Path('./Data')
CMV_pathlib = CMV_data_folder / 'VDJdb_CMV_TCR.tsv'
CMV_raw = read_data_VDJdb(str(CMV_pathlib))
CMV_raw = CMV_raw.drop_duplicates(ignore_index=True)
# Rearrange CMV_data
CMV_data = CMV_raw.loc[CMV_raw['Gene'] == 'TRB']
CMV_data = CMV_data.rename(columns={'cdr3': 'cdr3b'})
CMV_cdr3b = CMV_data[['cdr3b']]
CMV_cdr3b = CMV_cdr3b.drop_duplicates(ignore_index=True)

In [48]:
CMV_cdr3b.shape #4421 entries

(4421, 1)

### Find shared CDR3b of CMV with COVID-19

In [50]:
CMV_Common = pd.DataFrame(columns=data_all.columns.values.tolist())
for index, row in MIRA_data.iterrows():
    if row['cdr3b'] in CMV_cdr3b['cdr3b'].values:
        CMV_Common = CMV_Common.append(row, ignore_index=True)

In [52]:
len(pd.unique(CMV_Common['cdr3b']))

274

### Write to file

In [57]:
CMV_Common.to_csv(str(MIRA_data_folder / "Output/CMV_Common.csv"), index=False)

### Appearance of TSCAN epitopes in shared CDR3b binding targets

In [59]:
pd.DataFrame(search_TSCAN(CMV_Common,TSCAN_epitope),columns = ['Frequency'],index = TSCAN_epitope)

,Frequency
LLLDRLNQL,1
YLQPRTFLL,3
KLWAQCVQL,0
ALWEIQQVV,0


## Cross Reference with MART-1

### Read MART-1 data

In this MART1 dataset, epiotpe is only ELAGIGILTV. All cdrs are from HomoSapiens (human).
All are HLA-A02 cdrs. All are MHC class I.
format is ['Gene', 'cdr3', 'V', 'J']

In [60]:
MART1_data_folder = Path('./Data')
MART1_pathlib = MART1_data_folder / 'VDJdb_MART1_TCR.tsv'
MART1_raw = read_data_VDJdb(str(MART1_pathlib))
MART1_raw = MART1_raw.drop_duplicates(ignore_index=True)
# Rearrange MART1_data
MART1_data = MART1_raw.loc[MART1_raw['Gene'] == 'TRB']
MART1_data = MART1_data.rename(columns={'cdr3': 'cdr3b'})
MART1_cdr3b = MART1_data[['cdr3b']]
MART1_cdr3b = MART1_cdr3b.drop_duplicates(ignore_index=True)

In [61]:
MART1_cdr3b.shape

(1409, 1)

### Find shared CDR3b of MART1 with COVID-19

In [62]:
MART1_Common = pd.DataFrame(columns=data_all.columns.values.tolist())
for index, row in MIRA_data.iterrows():
    if row['cdr3b'] in MART1_cdr3b['cdr3b'].values:
        MART1_Common = MART1_Common.append(row, ignore_index=True)

In [63]:
len(pd.unique(MART1_Common['cdr3b']))

89

### Write to file

In [64]:
MART1_Common.to_csv(str(MIRA_data_folder / "Output/MART1_Common.csv"), index=False)

### Appearance of TSCAN epitopes in shared CDR3b binding targets

In [66]:
pd.DataFrame(search_TSCAN(MART1_Common,TSCAN_epitope),columns = ['Frequency'],index = TSCAN_epitope)

,Frequency
LLLDRLNQL,1
YLQPRTFLL,1
KLWAQCVQL,0
ALWEIQQVV,0
